In [30]:
import pandas as pd
import re


def read_tbl():
    df = pd.read_excel("../data/input/ms_response_copyofpostrosterfile.xlsx")

    df = df.rename(columns={"FIRST": "first_name", 
    "LAST": "last_name",
    "STATUS": "employment_status", 
    "DOH": "start_date", 
    "DOT": "end_date",
    "CERT #": "person_nbr"})
    return df 

def filter_uid(df):
    # Create number pattern for 4 or 5 digits
    pattern = r'^\d{4,5}$'
    
    # Filter rows where cert matches pattern
    mask = df['person_nbr'].astype(str).str.match(pattern)
    
    return df[mask]


def fix_agency_name(df):
    df.loc[:, "TYPE"] = df.TYPE.str.replace(r"(\w+) $", r"\1", regex=True)
    df.loc[:, "agency_name"] = df.Agency.str.cat(df.TYPE, sep=" ")
    return df



def clean_dates(df):
        df['start_date'] = pd.to_datetime(df['start_date'], errors="coerce")
        df['end_date'] =  pd.to_datetime(df['end_date'], errors="coerce")

        
        df.loc[:, "start_date"] = df.start_date.astype(str).str.lower().str.replace(r"nat", "", regex=True)
        df.loc[:, "end_date"] = df.end_date.astype(str).str.lower().str.replace(r"nat", "", regex=True)
        return df[~((df.start_date == ""))]



def clean_agency_name(df):
     df.loc[:, "agency_name"] = (df
                                 .agency_name
                                 .str.lower()
                                 .str.strip()
                                 .str.replace(r" (so|sheriff)$", " sheriff's office", regex=True)
                                 .str.replace(r" pd$", " police department", regex=True)
                                 .str.replace(r"(\w+)\s\+(\w+)", r"\1 \2", regex=True)
                                 .str.replace(r" dist ", " district ", regex=True)
                                 .str.replace(r" co ", " county ", regex=False)
                                 .str.replace(r"(\w+)  (\w+)", r"\1 \2", regex=True)
                                 .str.replace(r" off$", " office", regex=True)
                                 .str.replace(r" dept ", " department ", regex=False)
                                 .str.replace(r"lev brd", "levee board", regex=False)
     )
     return df[~((df.agency_name.fillna("").str.contains("fire")))]


def drop_cols(df):
     df = df.drop(columns=["Agency", "TYPE"])
df = read_tbl()

df = df.pipe(filter_uid).pipe(fix_agency_name).pipe(clean_dates).pipe(clean_agency_name)

df

/opt/homebrew/lib/python3.12/site-packages/pandas/core/indexing.py:1951: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[selected_item_labels] = value
/opt/homebrew/lib/python3.12/site-packages/pandas/core/indexing.py:1667: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[key] = value
/var/folders/r9/3_1rmy995xs_9z4vz66rsf9r0000gn/T/ipykernel_11656/1826697596.py:34: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer

,first_name,last_name,Agency,TYPE,employment_status,start_date,end_date,person_nbr,agency_name
1,Timothy,Pruitt,Abbeville,Chief,Part-Time,2010-04-01,,13792,abbeville chief
2,Jimmy,Easley,Abbeville,Chief,Part-Time,1995-05-15,2009-11-30,14876,abbeville chief
3,Jacob,Abel,ABC,State,Full-Time,2022-05-02,,19785,abc state
4,Kurt,Alexander,ABC,State,Full-Time,2010-09-13,,21028,abc state
5,William,Alexander,ABC,State,Full-Time,2002-07-01,2019-09-03,15761,abc state
...,...,...,...,...,...,...,...,...,...
45233,Stephen,James,Yazoo Levee Board,Lev Brd,Full-Time,2001-12-03,2006-11-09,15583,yazoo levee board levee board
45235,Bobby,Reed,Yazoo Levee Board,Lev Brd,Full-Time,2006-06-16,2014-09-30,3449,yazoo levee board levee board
45236,LaShun,Sims,Yazoo Levee Board,Lev Brd,Full-Time,2017-10-01,,26083,yazoo levee board levee board
45237,Reginald,Sims,Yazoo Levee Board,Lev Brd,Full-Time,2005-11-01,2021-06-10,18592,yazoo levee board levee board


In [ ]:
## done


df.to_csv("../data/output/mississippi-processed.csv", index=False)